In [16]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import xlrd
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric
import gensim
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
import nltk

In [ ]:
#!pip install xlrd

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    if doc is not None:
      stop_free = " ".join([i for i in str(doc).lower().split() if i not in stop])
      punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
      normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
      return normalized
    else:
      return doc

In [4]:
file_path = ("datasetds.xlsx")
data = pd.read_excel(file_path, 
                              sheet_name = 'All common workers from both ')

In [5]:
electric_data = data['Description']
doc_clean = [clean(doc).split() for doc in electric_data] 
dictionary = corpora.Dictionary(doc_clean) 
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean] 

In [9]:
base_model_lda = gensim.models.ldamodel.LdaModel
ldamodel = base_model_lda(doc_term_matrix, 
                          id2word = dictionary, passes=50)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [17]:
def predict(new_text):
  if new_text is not None and str(new_text).strip() != '':
    cleaned_data = clean(new_text).split()
    bow_vector = dictionary.doc2bow(cleaned_data)
    filters = [lambda x: x.lower(), strip_punctuation, strip_numeric]
    for index, score in sorted(ldamodel[bow_vector], key=lambda tup: -1*tup[1]):
      res = preprocess_string(ldamodel.print_topic(index, 5), filters)
      return res
  else:
    return None

In [18]:
for d in electric_data:
  print(predict(d))

['electrical', 'make', 'sure', 'service', 'project']
['provide', 'qualified', 'cooling', 'service', 'area']
['work', 'electrical', 'car', 'certificate', 'also']
['electrical', 'done', 'energy', 'work', 'melbourne']
None
None
['service', 'business', 'tony', 'project', 'maintenance']
['year', 'best', 'quality', 'work']
['electrical', 'installation', 'lighting', 'power', 'upgrade']
['year', 'microsoft', 'google', 'business', 'work']
None
['electrical', 'group', 'insured', 'industry', 'work']
['electrical', 'work', 'working', 'home', 'design']
['electrical', 'installation', 'lighting', 'power', 'upgrade']
['electrical', 'installation', 'lighting', 'power', 'upgrade']
['electrical', 'installation', 'lighting', 'power', 'upgrade']
['electrical', 'clean', 'work', 'professional', 'quality']
['electrician', 'year', 'experience', 'qualified', 'electrical']
['electrical', 'team', 'service', 'fully', 'power']
['electrical', 'service', 'work', 'need', 'customer']
['small', 'big', 'job', 'electrical

In [ ]:
coherence_model_lda = CoherenceModel(model=ldamodel, 
                                     texts=doc_clean, 
                                     dictionary=dictionary, 
                                     coherence="c_v")
coherence_lda = coherence_model_lda.get_coherence()
print('\nLDA Coherence Score: ', coherence_lda)


LDA Coherence Score:  0.43601596639572765


In [ ]:
lda_model_multi = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                                       id2word=dictionary,
                                       num_topics=2, 
                                       random_state=100,
                                       chunksize=100,
                                       alpha=0.01,
                                       eta=0.01,
                                       passes=5,
                                       per_word_topics=True)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [ ]:
coherence_model_multicore = CoherenceModel(model=lda_model_multi, 
                                     texts=doc_clean, 
                                     dictionary=dictionary, 
                                     coherence="c_v")
coherence_multicore = coherence_model_multicore.get_coherence()
print('\nMulticore Coherence Score: ', coherence_multicore)


Multicore Coherence Score:  0.5725416816704056
